2022/11/10<br>
This code is for matching FLM and polscope image by SIFT.<br>
Resolution of FLM is slightly higher than that of polscope.<br>
After matching, MFA of each cell can be freely accessible.<br>

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import cv2
import copy
import itertools
from module.im_stitching import *
from module.Stitcher import Stitcher
from skimage.filters import threshold_otsu, threshold_triangle, threshold_yen, threshold_mean, threshold_minimum, threshold_li, threshold_isodata
from skimage.util import img_as_ubyte
from tqdm import tqdm

Load all results

In [8]:
current_path=os.getcwd()
npz_path=os.path.join(current_path, "npz_file")
FLM_path=os.path.join(npz_path, "FLM_stitching_results")
MFA_path=os.path.join(npz_path, "stitching_results")
save_path=os.path.join(current_path, "fig_save", "image_registration")

if os.path.exists(save_path)==False:
    os.makedirs(save_path)

In [9]:
FLM_file_path_list=[]
MFA_file_path_list=[]
sample_name_list=[]

for root, dirs, files in os.walk(FLM_path):
    if (len(files)>0):
        file_path_extract=[os.path.join(root, files[i]) for i in range(len(files)) if "DS_Store" not in files[i]]
        FLM_file_path_list.append(file_path_extract)
    else:
        pass
    
for root, dirs, files in os.walk(MFA_path):
    if (len(files)>0):
        file_path_extract=[os.path.join(root, files[i]) for i in range(len(files)) if "DS_Store" not in files[i]]
        MFA_file_path_list.append(file_path_extract)
    else:
        pass

FLM_file_path_list=list(itertools.chain.from_iterable(FLM_file_path_list))
MFA_file_path_list=list(itertools.chain.from_iterable(MFA_file_path_list))

In [10]:
#extract common sample data from FLM and MFA
FLM_sample_list=[FLM_file_path_list[i].split("/")[-2] for i in range(len(FLM_file_path_list))]
MFA_sample_list=[MFA_file_path_list[i].split("/")[-2][:-2] for i in range(len(MFA_file_path_list))]

common_sample_list=list(set(FLM_sample_list) & set(MFA_sample_list))

FLM_file_path_list=[FLM_file_path for FLM_file_path in FLM_file_path_list if FLM_file_path.split("/")[-2] in common_sample_list]
MFA_file_path_list=[MFA_file_path for MFA_file_path in MFA_file_path_list if MFA_file_path.split("/")[-2][:-2] in common_sample_list]
"""
#binarization  condition
thres_const_list=np.array([0.5, 0.5, 0.5, 0.5, 1, 0.3, 1, 1, 0.3, 1, 
                           1, 1, 1, 1, 1, 1, 1, 1, 0.3, 1, 
                           1, 0.3, 1, 1, 1, 1, 0.3, 1, 1, 1, 
                           1, 1, 1, 1])
    

print(len(MFA_file_path_list)==len(thres_const_list))
"""

'\n#binarization  condition\nthres_const_list=np.array([0.5, 0.5, 0.5, 0.5, 1, 0.3, 1, 1, 0.3, 1, \n                           1, 1, 1, 1, 1, 1, 1, 1, 0.3, 1, \n                           1, 0.3, 1, 1, 1, 1, 0.3, 1, 1, 1, \n                           1, 1, 1, 1])\n    \n\nprint(len(MFA_file_path_list)==len(thres_const_list))\n'

Image registration

In [14]:
FLM_list=[]
MFA_list=[]

for FLM_file_path, MFA_file_path in tqdm(zip(FLM_file_path_list[9:10], MFA_file_path_list[9:10])):
    #unpack npz file
    npz_FLM=np.load(FLM_file_path, allow_pickle=True)
    npz_MFA=np.load(MFA_file_path, allow_pickle=True)
    
    FLM=npz_FLM["FLM"][:, :, 2]
    ret=np.nan_to_num(npz_MFA["ret_im"])
    angle=np.nan_to_num(npz_MFA["angle_im"])
    MFA=np.nan_to_num(npz_MFA["MFA_im"])
    sample_info=npz_MFA["sample_info"]
    
    
    #trim image bg beforehand
    #delete background as long as possible
    delete_line_h = np.where(np.all(ret == 0, axis=0))[0]
    delete_line_v = np.where(np.all(ret == 0, axis=1))[0]
    
    #ret
    ret = np.delete(ret, delete_line_h, axis=1)
    ret = np.delete(ret, delete_line_v, axis=0)
    
    #angle
    angle = np.delete(angle, delete_line_h, axis=1)
    angle = np.delete(angle, delete_line_v, axis=0)
    
    #MFA
    MFA = np.delete(MFA, delete_line_h, axis=1)
    MFA = np.delete(MFA, delete_line_v, axis=0)
    

    #binarization of flm and ret im
    threshold = threshold_otsu(ret)
    ret_binary=np.where(ret>threshold*0.4, 255, 0)
    threshold = threshold_otsu(FLM)
    FLM_binary=np.where(FLM>threshold, 255, 0)
    
    #matching ret and flm im
    stitcher=Stitcher()
    ret_binary_adj, FLM_binary_adj = image_dimension_adj_center(ret_binary, FLM_binary, const=0, criterion="B")
    _, FLM_adj = image_dimension_adj_center(ret_binary, FLM,const=0, criterion="B")
    ret_adj, _ = image_dimension_adj_center(ret, FLM, const=0, criterion="B")
    angle_adj, _ = image_dimension_adj_center(angle, FLM, const=0, criterion="B")
    MFA_adj, _ = image_dimension_adj_center(MFA, FLM, const=0, criterion="B")
    
    #image matching of ret and flm
    _, delete_line_h, delete_line_v, H=stitcher.stitch((np.uint8(ret_binary_adj), np.uint8(FLM_binary_adj)))

    #image matching of others under the same condition
    FLM_binary_matching=cv2.warpPerspective(np.uint8(FLM_binary_adj), H, (FLM_binary_adj.shape[1], 
                                                                          FLM_binary_adj.shape[0]), borderValue=255)
    FLM_matching=cv2.warpPerspective(FLM_adj, H, (FLM_adj.shape[1], FLM_adj.shape[0]), borderValue=0)
    
    
    #delete background as long as possible
    delete_line_h = np.where(np.all(ret_binary_adj == 0, axis=0))[0]
    delete_line_v = np.where(np.all(ret_binary_adj == 0, axis=1))[0]
    
    
    #FLM binary
    FLM_binary_trim = np.delete(FLM_binary_matching, delete_line_h, axis=1)
    FLM_binary_trim = np.delete(FLM_binary_trim, delete_line_v, axis=0)
    
    #FLM ori
    FLM_trim = np.delete(FLM_matching, delete_line_h, axis=1)
    FLM_trim = np.delete(FLM_trim, delete_line_v, axis=0)
    
    #ret
    ret_trim = np.delete(ret_adj, delete_line_h, axis=1)
    ret_trim = np.delete(ret_trim, delete_line_v, axis=0)
    
    #angle
    angle_trim = np.delete(angle_adj, delete_line_h, axis=1)
    angle_trim = np.delete(angle_trim, delete_line_v, axis=0)
    
    #MFA
    MFA_trim = np.delete(MFA_adj, delete_line_h, axis=1)
    MFA_trim = np.delete(MFA_trim, delete_line_v, axis=0)
    
    #result visualization
    fig, ax=plt.subplots(2, 2, figsize=(10, 8))
    im0=ax[0, 0].imshow(FLM_trim)
    pp=fig.colorbar(im0, ax=ax[0, 0])
    pp.mappable.set_clim(0, 255)
    
    im1=ax[0, 1].imshow(ret_trim, cmap="gist_ncar")
    pp=fig.colorbar(im1, ax=ax[0, 1])
    pp.mappable.set_clim(0, 137)
    
    im2=ax[1, 0].imshow(angle_trim, cmap="jet")
    pp=fig.colorbar(im2, ax=ax[1, 0])
    pp.mappable.set_clim(0, 180)
    
    im3=ax[1, 1].imshow(MFA_trim, cmap="gist_ncar")
    pp=fig.colorbar(im3, ax=ax[1, 1])
    pp.mappable.set_clim(0, 70)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, str(sample_info)+".png"), dpi=600)
    plt.close()
    
    #save result
    npz_save_path=os.path.join(npz_path, "image_registration", str(sample_info))
    if os.path.exists(npz_save_path)==False:
        os.makedirs(npz_save_path)

    np.savez_compressed(os.path.join(npz_save_path, str(sample_info)+".npz"), 
                       FLM=FLM_trim, FLM_binary=FLM_binary_trim, 
                       ret=ret_trim, angle=angle_trim, MFA=MFA_trim, 
                       sample_info=sample_info)
    
    #save for checking result
    FLM_list.append(FLM_trim)
    MFA_list.append(MFA_trim)

0it [00:00, ?it/s]

Size of original image, imageA: 4043*4387
Size of original image, imageB: 4043*4387
After concatenation, image: 4043*4387


1it [01:08, 68.72s/it]
